In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, '/home/samanti/Desktop/THESIS_LMT/imps/src')

import numpy as np
import open3d as o3d
import torch
from matplotlib import cm
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import jaccard_score
import pandas as pd

from imps.data import ScanNetScene, CLASS_NAMES
from imps.sqn.model import SQN
from imps.sqn.data_utils import prepare_input

SCENE_DIR1 = '/home/samanti/Desktop/data_practice/raw/scans/scene0040_00'
SCENE_DIR2 = '/home/samanti/Desktop/data_practice/raw/scans/scene0041_00'
SCENE_DIR3 = '/home/samanti/Desktop/data_practice/raw/scans/scene0009_00'
SCENE_DIR4 = '/home/samanti/Desktop/data_practice/raw/scans/scene0025_00'

N_POINTS = int(1.5e5)
# Not important we are not using this here yet. Keep this small for quick data processing
RESOLUTION = 25
SIGMAS = None
DEVICE = 'cpu'

IGNORED_LABELS = [0]

def get_iou(logits, labels):
    preds = np.argmax(logits, axis=-1)
    ious = []
    
    for c in range(len(CLASS_NAMES)):
        iou = jaccard_score((labels==c).astype(int), (preds==c).astype(int), pos_label=1)
        ious.append(iou)
        
    return np.array(ious)

In [2]:
# scene 1

scene1= ScanNetScene(SCENE_DIR1)
voxel_grid1, surface_points1, surface_colors1, vicinity_points1, vicinity_distances1, point_labels1 = scene1.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS
)

#For individual scenes:
#scene 1

class_counts1 = []
for c in range(len(CLASS_NAMES.keys())):
    class_counts1.append(np.sum(point_labels1 == c))
class_counts1 = np.array(class_counts1)

for ign in IGNORED_LABELS:
    class_counts1[ign] = 0
class_weights1 = class_counts1 / class_counts1.sum()


In [25]:
#scene 1 with model trained on single scene

features1 = torch.FloatTensor(surface_colors1).unsqueeze(0).to(DEVICE)
xyz1 = torch.FloatTensor(surface_points1).unsqueeze(0)

input_points1, input_neighbors1, input_pools1 = prepare_input(xyz1, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn1 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn1.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_without_mix3d'))
sqn1.eval();

In [4]:
#scene 1 with model trained on single scene

features1 = torch.FloatTensor(surface_colors1).unsqueeze(0).to(DEVICE)
xyz1 = torch.FloatTensor(surface_points1).unsqueeze(0)

input_points1, input_neighbors1, input_pools1 = prepare_input(xyz1, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn1 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn1.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation_vol_eldis'))
sqn1.eval();

In [3]:
#scene 1 with model trained on centered mixed scene

features1 = torch.FloatTensor(surface_colors1).unsqueeze(0).to(DEVICE)
xyz1 = torch.FloatTensor(surface_points1).unsqueeze(0)

input_points1, input_neighbors1, input_pools1 = prepare_input(xyz1, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn1 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn1.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation'))
sqn1.eval();

In [5]:
# scene 1

with torch.no_grad():
    all_logits1 = sqn1.forward(features1, input_points1, input_neighbors1, input_pools1, xyz1)
    all_logits1 = all_logits1.squeeze().detach().numpy()

In [6]:
#scene 1

df1 = pd.DataFrame()
df1['class'] = CLASS_NAMES.keys()
df1['iou'] = get_iou(all_logits1, point_labels1)
df1['weight'] = class_weights1
df1['iou_weighted'] = df1['iou'] * df1['weight']

print("mIOU for scene 1:", df1.iou_weighted.sum())
df1

mIOU for scene 1: 0.1437048855758032


/home/samanti/anaconda3/envs/mix3d/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,class,iou,weight,iou_weighted
0,unannotated,0.000329,0.000000,0.000000
1,wall,0.287267,0.303646,0.087228
2,floor,0.012294,0.279683,0.003439
3,cabinet,0.000000,0.059220,0.000000
4,bed,0.000000,0.000000,0.000000
5,chair,0.103453,0.053227,0.005506
6,sofa,0.000000,0.049416,0.000000
7,table,0.025860,0.013677,0.000354
8,door,0.000000,0.041817,0.000000
9,window,0.000000,0.022252,0.000000


In [7]:
#scene 1

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(surface_points1)
pcd1.colors = o3d.utility.Vector3dVector(scene1.colorize_labels(all_logits1.argmax(axis=-1)))

o3d.visualization.draw_geometries([pcd1]) 

3
(150000, 3)


In [8]:
# scene 2

scene2 = ScanNetScene(SCENE_DIR2)
voxel_grid2, surface_points2, surface_colors2, vicinity_points2, vicinity_distances2, point_labels2 = scene2.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS
)

#For individual scenes:
#scene 2

class_counts2 = []
for c in range(len(CLASS_NAMES.keys())):
    class_counts2.append(np.sum(point_labels2 == c))
class_counts2 = np.array(class_counts2)

for ign in IGNORED_LABELS:
    class_counts2[ign] = 0
class_weights2 = class_counts2 / class_counts2.sum()



In [4]:
# single scene 2 with model trained on single scene

features2 = torch.FloatTensor(surface_colors2).unsqueeze(0).to(DEVICE)
xyz2 = torch.FloatTensor(surface_points2).unsqueeze(0)

input_points2, input_neighbors2, input_pools2 = prepare_input(xyz2, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn2 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn2.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_without_mix3d'))
sqn2.eval();

In [8]:
# single scene 2 with model trained on centered mixed scene
features2 = torch.FloatTensor(surface_colors2).unsqueeze(0).to(DEVICE)
xyz2 = torch.FloatTensor(surface_points2).unsqueeze(0)

input_points2, input_neighbors2, input_pools2 = prepare_input(xyz2, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn2 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn2.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation'))
sqn2.eval();

In [9]:
# single scene 2 with model trained on centered mixed scene
features2 = torch.FloatTensor(surface_colors2).unsqueeze(0).to(DEVICE)
xyz2 = torch.FloatTensor(surface_points2).unsqueeze(0)

input_points2, input_neighbors2, input_pools2 = prepare_input(xyz2, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn2 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn2.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation_vol_eldis'))
sqn2.eval();

In [10]:
# scene 2

with torch.no_grad():
    all_logits2 = sqn2.forward(features2, input_points2, input_neighbors2, input_pools2, xyz2)
    all_logits2 = all_logits2.squeeze().detach().numpy()

In [11]:
# scene 2

df2 = pd.DataFrame()
df2['class'] = CLASS_NAMES.keys()
df2['iou'] = get_iou(all_logits2, point_labels2)
df2['weight'] = class_weights2
df2['iou_weighted'] = df2['iou'] * df2['weight']

print("mIOU for scene 2:", df2.iou_weighted.sum())
df2

mIOU for scene 2: 0.05189441345197856


/home/samanti/anaconda3/envs/mix3d/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samanti/anaconda3/envs/mix3d/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samanti/anaconda3/envs/mix3d/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samanti/anaconda3/envs/mix3d/lib/p

,class,iou,weight,iou_weighted
0,unannotated,0.000000,0.000000,0.000000
1,wall,0.202739,0.191040,0.038731
2,floor,0.011681,0.470563,0.005496
3,cabinet,0.000000,0.000000,0.000000
4,bed,0.000000,0.000000,0.000000
5,chair,0.045746,0.167594,0.007667
6,sofa,0.000000,0.000000,0.000000
7,table,0.000000,0.154448,0.000000
8,door,0.000000,0.011931,0.000000
9,window,0.000000,0.004425,0.000000


In [12]:
#scene 2

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(surface_points2)
pcd2.colors = o3d.utility.Vector3dVector(scene2.colorize_labels(all_logits2.argmax(axis=-1)))

o3d.visualization.draw_geometries([pcd2]) 

3
(150000, 3)


In [13]:
# scene 3

scene3= ScanNetScene(SCENE_DIR3)
voxel_grid3, surface_points3, surface_colors3, vicinity_points3, vicinity_distances3, point_labels3 = scene3.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS
)

#scene 3

class_counts3 = []
for c in range(len(CLASS_NAMES.keys())):
    class_counts3.append(np.sum(point_labels3 == c))
class_counts3 = np.array(class_counts3)

for ign in IGNORED_LABELS:
    class_counts3[ign] = 0
class_weights3 = class_counts3 / class_counts3.sum()



In [6]:
# single scene 3 with model trained on single scene

features3 = torch.FloatTensor(surface_colors3).unsqueeze(0).to(DEVICE)
xyz3 = torch.FloatTensor(surface_points3).unsqueeze(0)


input_points3, input_neighbors3, input_pools3 = prepare_input(xyz3, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn3 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn3.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_without_mix3d'))
sqn3.eval();

In [13]:
# single scene 3 with model trained on centered mixed scene

features3 = torch.FloatTensor(surface_colors3).unsqueeze(0).to(DEVICE)
xyz3 = torch.FloatTensor(surface_points3).unsqueeze(0)
input_points3, input_neighbors3, input_pools3 = prepare_input(xyz3, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn3 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn3.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation'))
sqn3.eval();

In [14]:
# single scene 3 with model trained on centered mixed scene

features3 = torch.FloatTensor(surface_colors3).unsqueeze(0).to(DEVICE)
xyz3 = torch.FloatTensor(surface_points3).unsqueeze(0)
input_points3, input_neighbors3, input_pools3 = prepare_input(xyz3, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn3 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn3.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation_vol_eldis'))
sqn3.eval();

In [15]:
# scene 3

with torch.no_grad():
    all_logits3 = sqn3.forward(features3, input_points3, input_neighbors3, input_pools3, xyz3)
    all_logits3 = all_logits3.squeeze().detach().numpy()

In [16]:
# scene 3

df3 = pd.DataFrame()
df3['class'] = CLASS_NAMES.keys()
df3['iou'] = get_iou(all_logits3, point_labels3)
df3['weight'] = class_weights3
df3['iou_weighted'] = df3['iou'] * df3['weight']

print("mIOU3:", df3.iou_weighted.sum())
df3

mIOU3: 0.34510805763261276


/home/samanti/anaconda3/envs/mix3d/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samanti/anaconda3/envs/mix3d/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,class,iou,weight,iou_weighted
0,unannotated,0.000000,0.000000,0.000000
1,wall,0.586623,0.578953,0.339627
2,floor,0.014197,0.386098,0.005481
3,cabinet,0.000000,0.000000,0.000000
4,bed,0.000000,0.000000,0.000000
5,chair,0.000000,0.000000,0.000000
6,sofa,0.000000,0.000000,0.000000
7,table,0.000000,0.000000,0.000000
8,door,0.000000,0.000000,0.000000
9,window,0.000000,0.000000,0.000000


In [17]:
#scene 3

pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(surface_points3)
pcd3.colors = o3d.utility.Vector3dVector(scene3.colorize_labels(all_logits3.argmax(axis=-1)))

o3d.visualization.draw_geometries([pcd3]) 

3
(150000, 3)


In [18]:
# scene 4

scene4 = ScanNetScene(SCENE_DIR4)
voxel_grid4, surface_points4, surface_colors4, vicinity_points4, vicinity_distances4, point_labels4 = scene4.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS
)


#scene 4

class_counts4 = []
for c in range(len(CLASS_NAMES.keys())):
    class_counts4.append(np.sum(point_labels4 == c))
class_counts4 = np.array(class_counts4)

for ign in IGNORED_LABELS:
    class_counts4[ign] = 0
class_weights4 = class_counts4 / class_counts4.sum()


In [4]:
# single scene 4 with model trained with single scene

features4 = torch.FloatTensor(surface_colors4).unsqueeze(0).to(DEVICE)
xyz4 = torch.FloatTensor(surface_points4).unsqueeze(0)

input_points4, input_neighbors4, input_pools4 = prepare_input(xyz4, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn4 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn4.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_without_mix3d'))
sqn4.eval();

In [18]:
# single scene 4 with model trained with centered mixed scene

features4 = torch.FloatTensor(surface_colors4).unsqueeze(0).to(DEVICE)
xyz4 = torch.FloatTensor(surface_points4).unsqueeze(0)


input_points4, input_neighbors4, input_pools4 = prepare_input(xyz4, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn4 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn4.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation'))
sqn4.eval();

In [19]:
# single scene 4 with model trained with centered mixed scene

features4 = torch.FloatTensor(surface_colors4).unsqueeze(0).to(DEVICE)
xyz4 = torch.FloatTensor(surface_points4).unsqueeze(0)


input_points4, input_neighbors4, input_pools4 = prepare_input(xyz4, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)

sqn4 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
sqn4.load_state_dict(torch.load('/home/samanti/Desktop/data_practice/saved/mix3d_training_losss_vs_epoch/SQN/sqn_with_mix3d_with_augmentation_vol_eldis'))
sqn4.eval();

In [20]:
# scene 4

with torch.no_grad():
    all_logits4 = sqn4.forward(features4, input_points4, input_neighbors4, input_pools4, xyz4)
    all_logits4 = all_logits4.squeeze().detach().numpy()

In [21]:
# scene 4

df4 = pd.DataFrame()
df4['class'] = CLASS_NAMES.keys()
df4['iou'] = get_iou(all_logits4, point_labels4)
df4['weight'] = class_weights4
df4['iou_weighted'] = df4['iou'] * df4['weight']

print("mIOU4:", df4.iou_weighted.sum())
df4

mIOU4: 0.2312724654526345


/home/samanti/anaconda3/envs/mix3d/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,class,iou,weight,iou_weighted
0,unannotated,0.000000,0.000000,0.000000
1,wall,0.447917,0.469029,0.210086
2,floor,0.000000,0.154347,0.000000
3,cabinet,0.000000,0.077260,0.000000
4,bed,0.000000,0.000000,0.000000
5,chair,0.003545,0.027677,0.000098
6,sofa,0.000000,0.096227,0.000000
7,table,0.000000,0.000000,0.000000
8,door,0.315403,0.064464,0.020332
9,window,0.000000,0.019804,0.000000


In [22]:
#scene 4

pcd4 = o3d.geometry.PointCloud()
pcd4.points = o3d.utility.Vector3dVector(surface_points4)
pcd4.colors = o3d.utility.Vector3dVector(scene4.colorize_labels(all_logits4.argmax(axis=-1)))

o3d.visualization.draw_geometries([pcd4]) 

3
(150000, 3)
